In [1]:
from config import SCRIPT_DIR, OUTPUT_DIR
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

In [4]:
drop_cols = ['label', 'Ticker', 'Date', 'Ret_1d']

## Load data and finalize the preparations for PCA
path = os.path.join(SCRIPT_DIR, './data/sp500_features.csv')
df = pd.read_csv(path)
df = df.dropna()
df = df.sort_values('Date')

,Date,Ticker,Adj Close,Ret_1d,Ret_5d,Ret_20d,Ret_60d,Ret_120d,Vol_10d,Vol_20d,Vol_60d,MACD,MACD_Signal,RSI14
5895,2021-01-04,A,115.095314,0.002908,0.012996,0.046263,0.143960,0.312701,0.006020,0.010008,0.013854,1.454728,1.718475,51.899159
443425,2021-01-04,OXY,16.627232,0.008088,-0.012450,0.065231,0.698359,0.047736,0.035054,0.052102,0.055674,0.528805,0.912887,29.376553
443424,2021-01-04,OTIS,61.415703,-0.029016,-0.003040,0.001527,0.011337,0.150379,0.012875,0.014255,0.013955,0.298241,0.210925,59.171524
443423,2021-01-04,ORCL,59.565727,-0.014531,-0.018627,0.075587,0.052154,0.119023,0.010609,0.010993,0.011142,1.486610,1.537787,69.674231
443422,2021-01-04,ON,32.669998,-0.001833,0.021257,0.122680,0.399143,0.579024,0.009544,0.013465,0.021991,1.047631,1.074329,78.470210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62805,2025-07-17,ADP,302.119995,0.006161,-0.012099,-0.017879,0.065928,0.031939,0.009388,0.013780,0.011021,-2.550001,-2.040010,48.798191
62804,2025-07-17,ADM,53.540001,-0.008519,-0.024595,-0.010351,0.124765,0.079650,0.009729,0.014818,0.015381,0.967316,1.234251,51.859311
62803,2025-07-17,ADI,240.970001,0.001496,-0.016971,0.059488,0.383320,0.064683,0.009424,0.009826,0.018979,4.822515,5.869061,56.958667
62816,2025-07-17,ALGN,192.270004,0.010671,-0.059712,0.097682,0.116421,-0.181447,0.018236,0.015826,0.020941,3.431494,4.128132,55.761896
